In [1]:
cd ../python_scripts/

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts


In [2]:
from uszipcode import SearchEngine

In [3]:
from datetime import datetime
import os
import pandas as pd
from pycountry import countries
from utils import AddDailyFields

In [4]:
import utils
from nyt_us_data import NYTCountyCases
from data_aggregators import (GlobalDataJHU, USDataNYT)


In [5]:
pd.options.display.max_rows=500
pd.options.display.max_columns=500

In [12]:
search = SearchEngine(simple_zipcode=True)

In [6]:
yay = USDataNYT()
yay.run()

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/utils.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df["previous_days_date"] = df["date"] + pd.np.timedelta64(1, "D")


Zero day for states
Zero day for states
Zero day for states
Zero day for states
Adding in country codes


/Users/amlvt225/anaconda3/envs/osh_v2/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [7]:
us = yay.data

In [9]:
def pandas_apply_location_attr(row, attr='zipcode'):
    """
    Pandas function to loop thru a df, grab lat and long
    values, then search for a specific attribute in the
    zip class
    """
    lat = row['lat']
    long = row['long']
    result = lat_long_lookup(lat, long)
    try:
        return getattr(result, attr)
    except AttributeError:
        return "NOT FOUND"

def lat_long_lookup(lat, long):
    """
    Arguments: Latitude, Longitude
    Returns: a "ZIPCODE" object
    """
    result = search.by_coordinates(lat, long, radius=40, returns=1)
    try:
        trimmed = result[0]
    except IndexError:
        print(lat, long)
        return "NOT FOUND"
    
    return trimmed
    #return result.zipcode

In [13]:
lat_long_lookup(35.308272, -79.492725)

SimpleZipcode(zipcode='27376', zipcode_type='Standard', major_city='West End', post_office_city='West End, NC', common_city_list=['West End', 'Seven Lakes'], county='Moore County', state='NC', lat=35.25, lng=-79.53, timezone='Eastern', radius_in_miles=7.0, area_code_list=['910'], population=9386, population_density=135.0, land_area_in_sqmi=69.76, water_area_in_sqmi=1.87, housing_units=4418, occupied_housing_units=3908, median_home_value=224700, median_household_income=53877, bounds_west=-79.641693, bounds_east=-79.435784, bounds_north=35.323311, bounds_south=35.181445)

In [14]:
ref_lat_long = us[['state_code','county','lat','long']].drop_duplicates()

In [15]:
ref_lat_long['TEST_zipcode'] = ref_lat_long.apply(pandas_apply_location_attr, 
                                                  axis=1,
                                                  attr='zipcode')

nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
35.829692 -111.773728
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
62.27909399999999 -149.619094
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan


In [16]:
ref_lat_long['TEST_county'] = ref_lat_long.apply(pandas_apply_location_attr, 
                                                  axis=1,
                                                  attr='county')

nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
35.829692 -111.773728
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
62.27909399999999 -149.619094
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan


In [17]:
ref_lat_long['TEST_state'] = ref_lat_long.apply(pandas_apply_location_attr, 
                                                  axis=1,
                                                  attr='state')

nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
35.829692 -111.773728
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
62.27909399999999 -149.619094
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan


In [18]:
ref_lat_long['TEST_population'] = ref_lat_long.apply(pandas_apply_location_attr, 
                                                  axis=1,
                                                  attr='population')

nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
35.829692 -111.773728
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
62.27909399999999 -149.619094
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan


In [27]:
ref_lat_long['TEST_population_density'] = ref_lat_long.apply(pandas_apply_location_attr, 
                                                  axis=1,
                                                  attr='population_density')

nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
35.829692 -111.773728
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
62.27909399999999 -149.619094
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan


In [32]:
ref_lat_long.head()

,state_code,county,lat,long,TEST_zipcode,TEST_county,TEST_state,TEST_population,TEST_population_density
0,WA,Snohomish,48.070464,-121.936784,98258,Snohomish County,WA,30524,1058
3,IL,Cook,41.894294,-87.645455,60654,Cook County,IL,14875,24468
5,CA,Orange,33.675687,-117.777207,92604,Orange County,CA,26853,7960
8,AZ,Maricopa,33.345176,-112.498930,85338,Maricopa County,AZ,41115,557
9,CA,Los Angeles,34.196398,-118.261862,91208,Los Angeles County,CA,16245,2354


In [39]:
matches = ref_lat_long.loc[ref_lat_long['state_code'] == ref_lat_long['TEST_state']]

In [40]:
misses = ref_lat_long.loc[ref_lat_long['state_code'] != ref_lat_long['TEST_state']]

In [41]:
matches.shape, misses.shape

((1767, 9), (30, 9))

In [42]:
pwd

'/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts'

In [46]:
ref_lat_long.to_csv('../ref_data/county_to_zip_ref.csv', index=False)

In [45]:
pwd

'/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts'

In [ ]:
cd ../output_data/cou